In [1]:
pip install nltk


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install contractions


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:

import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import re
from numpy import array, average
import nltk
nltk.download('stopwords')
import nltk
nltk.download('omw-1.4')
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# NLP Assignment 1 (40% of grade): Text classification for Fake News Detection

This coursework will involve you implementing functions for a text classifier, which you will train to detect **fake news** in a corpus of approx. 12,000 statements. 

In this template you are given the basis for that implementation, though some of the functions are missing, which you have to fill in.

Follow the instructions file **NLP_Assignment_1_Instructions.pdf** for details of each question - the outline of what needs to be achieved for each question is as below.

You must submit all **ipython notebooks and extra resources you need to run the code if you've added them** in the code submission, and a **2 page report (pdf)** in the report submission on QMPlus where you report your methods and findings according to the instructions file for each question.

# Question 1: Input and Basic preprocessing (10 marks)

In [4]:
from scipy.sparse import data
df_fn = pd.read_csv('fake_news.tsv',sep='\t') # Load TSV file
tuple_list = []
def convert_label(label):
    """Converts the multiple classes into two,
    making it a binary distinction between fake news and real."""
    #return label
    # Converting the multiclass labels to binary label
    labels_map = {
        'true': 'REAL',
        'mostly-true': 'REAL',
        'half-true': 'REAL',
        'false': 'FAKE',
        'barely-true': 'FAKE',
        'pants-fire': 'FAKE'
    }
    return labels_map[label]

data_line_list = df_fn.iloc[:,1] # Select list of all Datalines in line/label columns


def parse_data_line(data_line_list):
  # Should return a tuple of the label as just FAKE or REAL and the statement
  # e.g. (label, statement)

    data_label = data_line_list[0] # Select label column from the row
    data_line = data_line_list[1] + data_line_list[3] + data_line_list[4] + data_line_list[5] # Select feature column
    label = convert_label(data_label) # Convert label

    return (label, data_line) # Return tuple of label and feature

In [5]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
import contractions


num_tokens_text_dict = {}
text = "Is this, http://www.google.com : correcting feet won't?"

# Input: a string of one statement
def pre_process(text):
  # Add sylistic feature: count of unique words per statement. Achieved by just spliting the unprocessed text
  num_tokens_text_dict['number'] = len(text.split())    # Pass the length of tokens to dictionary 
  # ---------------------------------------------------- 
  # Tried and rejected methods
    
  # Remove Lowercase & URL Removal
  #tokens = re.sub(r'http\S+', '', text.lower())

  # Remove punctuation
  #tokens = re.sub(r'[^\w\s]', '', tokens)

  #------------------------------------------------------

  # Split contraction
  tokens = ' '.join([contractions.fix(word) for word in text.split()])
  
  # Tokenize
  tokens = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", tokens) # separates punctuation at ends of strings
  tokens = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\1 \2", tokens) # separates punctuation at beginning of strings
  tokens = re.split(r"\s+",tokens)

  

  # Convert to lower case
  tokens = [t.lower() for t in tokens]

  # Stop word removal 
  stop_words = set(stopwords.words('english')) # List of stopwords in 'english' to compare with 
  filtered_sentence = []

  for w in tokens:                                # For words in tokens 
    if w not in stop_words:                       # If w is not in stop_words list
        filtered_sentence.append(w)               # Append to new list variable
  tokens = filtered_sentence                      # Fresh list of tokens exclude stop words


  # Stemming
  # Create object of PorterStemmer library which removes suffixes or prefixes used with a word
  #port = PorterStemmer()                        
  #tokens = [port.stem(word) for word in tokens] # Pass each word through the stem method for stemming

  # Lemmitization
  # Create object of PorterStemmer library which converts the word to its base form
  lem = WordNetLemmatizer()
  tokens = [lem.lemmatize(words) for words in tokens] # Pass each word through the lemmatize method


  return tokens

# Test
#print("This is a preprocessed sentence: ", pre_process(text))
#print(num_tokens_text_dict)

In [6]:
def load_data(path):
   
    """Load data from a tab-separated file and append it to raw_data."""
    with open(path) as f:

        reader = csv.reader(f, delimiter='\t') # Read file
        for line in reader:                    # For each line in the file
          #print(line)
          if line[0] == "Id":                  # Skip header
              continue
          (label, text) = parse_data_line(line)# Return (label, text) from parse function
          raw_data.append((text, label))       # Append it to variable as raw data
    return raw_data


test1 = []
train1 = []

def split_and_preprocess_data(percentage):
    
    """Split the data between train_data and test_data according to the percentage
    and performs the preprocessing."""
    #print(raw_data[0:2])
    num_samples = len(raw_data) 
    #print(num_samples)
    num_training_samples = int((percentage * num_samples)) 
    #print(num_training_samples)
    
    # The text is first pre-proccessed and then converted to feature vector 
    for (text, label) in raw_data[:num_training_samples]: # for tuples in raw data from 0 to num train sample calculated
      train1.append((text, label))                        # For printing original text in .txt analysis file
      train_data.append((to_feature_vector(pre_process(text)),label)) # train data generated for training classifier
    for (text, label) in raw_data[num_training_samples:]: # test data
        test_data.append((to_feature_vector(pre_process(text)),label)) # test data generated for making predictions
   

# Question 2: Basic Feature Extraction (20 marks)

In [7]:
# Dictionary that has tokens for keys, and values a weight for each of those tokens in the preprocessed statements. 
from collections import Counter
global_feature_dict = {}

# Creates list of tokens using pre_processed func
# Pre-processed tokens passed through to_feature_vector function
def to_feature_vector(tokens):

  # Extract the value ({number: lenght}) of the dictionary to a list
  num_tokens_text = list(num_tokens_text_dict.values()) 

  #Global feature vector
  #------------------------------------------------------------------
  i=0
  for word in tokens:                           # For all words in a sentence
    if word not in global_feature_dict.keys():  # If word already in global dict
      if bool(global_feature_dict) == False:    # If dict is empty
        global_feature_dict[word] = 0           # Assign 0 to word
      else:                                     # Else assign value of last element +1 
        global_feature_dict[word] = list(global_feature_dict.values())[-1]+1  
      i+=1                                      # Next word
    else:
      i+=1                                      # If word already in global dict then next

  #-----------------------------------------------------------------
 
  ngrams = Counter() # a counter for how many times a given ngram sequence w_i-1,w_i occurs
  
  order = 3 # Order (i.e. n) of the language model (Tried with bigram and trigram)
  tokens = ['<s>'] * (order-1) + tokens + ['</s>'] # Add start and stop tags for ngram
  

  for i in range(order - 1, len(tokens)):  # For loop to get the ngram sequence 
      context = tokens[i-order+1:i]        # Sequence of context/(n-1)previous tokens
      target = tokens[i]                   # The token nth token
      ngram_sequence = context + [target]  # Final n-gram sequence is the combination of (n-1)context + nth target 
      
      ngrams[' '.join(ngram_sequence)] += 1  # Count how many times the sequence occurs
      ngram_dict = ngrams

  
  
  ngram_dict['No.of tokens'] = num_tokens_text[0]  # Add sylistic feature: Number of unique words per feature
  
  return ngram_dict

In [8]:
# Test to_feature vector function and global dictionary
text = ["lord mercy lord mercy lord mercy That old why is two it , is it as as finish","lord mercy what flower why"]

for text in text:
  print("weight",to_feature_vector(pre_process(text)))

print("Global:", global_feature_dict,"\n")


weight Counter({'No.of tokens': 18, 'lord mercy lord': 2, 'mercy lord mercy': 2, '<s> <s> lord': 1, '<s> lord mercy': 1, 'lord mercy old': 1, 'mercy old two': 1, 'old two ,': 1, 'two , finish': 1, ', finish </s>': 1})
weight Counter({'No.of tokens': 5, '<s> <s> lord': 1, '<s> lord mercy': 1, 'lord mercy flower': 1, 'mercy flower </s>': 1})
Global: {'lord': 0, 'mercy': 1, 'old': 2, 'two': 3, ',': 4, 'finish': 5, 'flower': 6} 



In [9]:
# TRAINING AND VALIDATING OUR CLASSIFIER
from sklearn.naive_bayes import MultinomialNB

def train_classifier(data):

    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC(class_weight={0:0.8,1:0.2},max_iter=10000))]) 
    # .train() method which takes a dict of featureset and labels as input
    return SklearnClassifier(pipeline).train(data) 

# Question 3: Cross-validation (20 marks)

In [10]:
#from traitlets.traitlets import validate
#solution
from sklearn.metrics import classification_report

def cross_validate(dataset, folds):
   
    results = []
    #results_avg = []
    #accuracy = 0.0
    fold_size = int(len(dataset)/folds) + 1
    
    
    for i in range(0,len(dataset),int(fold_size)):
        # insert code here that trains and tests on the 10 folds of data in the dataset
        print("Fold start on items %d - %d" % (i, i+fold_size))
        # FILL IN THE METHOD HERE
        
        # Validation fold of len(db)/10 + 1
        validation_fold = dataset[i:i+fold_size]  
        
        # train-fold which is all data expect val-fold
        train_fold= dataset[0:i] + dataset[(i+fold_size):len(dataset)] 
        # Train classifier for each fold
        classifier = train_classifier(train_fold) 
        
        y_pred =[]
        y_true = []
        for j in range(0, 812): # Range of val-fold

          y_pred.append((predict_labels(validation_fold[j][0], classifier))) #Predict labels for val-fold
          y_true.append(validation_fold[j][1]) #True labels of that val-fold
          
        #print("Total predicted labels:",len(y_pred))
        #print("Total true labels:", len(y_true))
        target_names = ['FAKE', 'REAL']  # For heatmap


        # Classification report
        a = precision_recall_fscore_support(y_true, y_pred, average='weighted') # Calculate P, r ,fscore
        accuracy = accuracy_score(y_true, y_pred) # Calculate accuracy
        p, r, f = a[:3]
        results.append((p, r, f, accuracy)) # Performance result for each fold
        

        result_array = array(results)
        # Calculate average of 'results'. (Axis=0: along the column)
        cv_results = list(average(result_array, axis=0).round(2)) 

        # Heatmap for error analysis
        #confusion_matrix_heatmap(y_true, y_pred, target_names)

        
    return results, cv_results

In [11]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predict_labels(samples, classifier):
    """Assuming preprocessed samples, return their predicted labels from the classifier model."""
    return classifier.classify_many(samples)


def predict_label_from_raw(sample, classifier):
    """Assuming raw text, return its predicted label from the classifier model."""
    return classifier.classify(to_feature_vector(pre_process(sample)))

In [12]:
# MAIN

# loading reviews
# initialize global lists that will be appended to by the methods below
raw_data = []          # the filtered data from the dataset file
train_data = []        # the pre-processed training data as a percentage of the total dataset
test_data = []         # the pre-processed test data as a percentage of the total dataset

df = pd.read_csv('fake_news.tsv',sep='\t') # Load TSV file 

# Clean file for NaN values
df['label'].fillna('0', inplace=True) 
df['statement'].fillna('0', inplace=True)   
df['subject'].fillna('0', inplace=True)   
df['speaker'].fillna('0', inplace=True)   
df['speaker_job_title'].fillna('0', inplace=True)   
df['state_info'].fillna('0', inplace=True)   
df['party_affiliation'].fillna(0, inplace=True)   
df['total_barely_true_counts'].fillna(0, inplace=True)  
df['total_false_counts total_half_true_counts'].fillna(0, inplace=True)  
df['total_mostly_true_counts'].fillna(0, inplace=True)  
df['total_pants_on_fire_counts'].fillna(0, inplace=True)  
df['context'].fillna('0', inplace=True) 


# New file created with no NaN values
df.to_csv("fake_news1.tsv", sep="\t",index=False)  

# references to the data files
data_file_path = 'fake_news1.tsv'

# Do the actual stuff (i.e. call the functions we've made)
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(raw_data), len(train_data), len(test_data)),
      "\nPreparing the dataset...",sep='\n')
print()

load_data(data_file_path) 

# We split the raw dataset into a set of training data and a set of test data (80/20)
# cross validation is done on the 80% (training data)
# We print the number of training samples and the number of features before the split
print("Now %d rawData, %d trainData, %d testData" % (len(raw_data), len(train_data), len(test_data)),
      "\nPreparing training and test data...",sep='\n')
print()

split_and_preprocess_data(0.8)

# We print the number of training samples and the number of features after the split
print("After split, %d rawData, %d trainData, %d testData" % (len(raw_data), len(train_data), len(test_data)),
      "Training Samples: ", len(train_data), "Features: ", len(global_feature_dict), sep='\n')


Now 0 rawData, 0 trainData, 0 testData

Preparing the dataset...

Now 10240 rawData, 0 trainData, 0 testData

Preparing training and test data...

After split, 10240 rawData, 8192 trainData, 2048 testData
Training Samples: 
8192
Features: 
16007


# 4. Error Analysis (10 marks)

In [13]:
from sklearn import metrics
import matplotlib.pyplot as plt

# a function to make the confusion matrix readable and pretty
def confusion_matrix_heatmap(y_test, preds, labels):
    """Function to plot a confusion matrix"""
    # pass labels to the confusion matrix function to ensure right order
    cm = metrics.confusion_matrix(y_test, preds, labels=labels)
    

    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Confusion matrix of the classifier')
    fig.colorbar(cax)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels( labels, rotation=45)
    ax.set_yticklabels( labels)

    for i in range(len(cm)):
        for j in range(len(cm)):
            #print(j, i, cm[i, j])
            text = ax.text(j, i, cm[i, j],
                           ha="center", va="center", color="w")

    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    # fix for mpl bug that cuts off top/bottom of seaborn viz:
    b, t = plt.ylim() # discover the values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    #plt.ylim(b, t) # update the ylim(bottom, top) values
    plt.show() # ta-da!
    plt.show()


In [ ]:
a, b = cross_validate(train_data, 10)  # will work and output overall performance of p, r, f-score when cv implemented

Fold start on items 0 - 820
Training Classifier...


In [ ]:
# Print Avg performance
print("Avg Precision: {0}\nAvg Recall: {1}\nAvg Fscore: {2}\nAvg Accuracy: {3}\n".format(b[0],b[1],b[2],b[3]))

# Print performance for each fold
#for i in range(0, len(a)):
    #print("Fold", i)
    #print("Precision: {0}, Recall: {1}, Fscore: {2}, Accuracy: {3}\n".format(a[i][0].round(2),a[i][1].round(2),a[i][2].round(2),a[i][3].round(2)))

# Questions 5 (20%) and 6 (20%) (recommend starting a new notebook)

In [ ]:
# Finally, check the accuracy of your classifier by training on all the traning data
# and testing on the test set
# Will only work once all functions are complete
functions_complete = True  # set to True once you're happy with your methods for cross val
if functions_complete:
    print(test_data[0])   # have a look at the first test data instance
    classifier = train_classifier(train_data)  # train the classifier
    test_true = [t[1] for t in test_data]   # get the ground-truth labels from the data
    test_pred = predict_labels([x[0] for x in test_data], classifier)  # classify the test data to get predicted labels
    final_scores = precision_recall_fscore_support(test_true, test_pred, average='weighted') # evaluate
    accuracy = accuracy_score(test_true, test_pred) # Calculate accuracy
    print("Done training!")
    print("Precision: %f\nRecall: %f\nF Score:%f" % final_scores[:3])
    print("Accuracy",accuracy)
   